In [11]:
import pandas as pd
import psycopg2

# Connect to your PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    dbname="moviedb",
    user="ajinkyaambadkar",
    password="Achiever216"  # Replace with your real PostgreSQL password
)

# Load ratings data
ratings_df = pd.read_sql("SELECT user_id, movie_id, rating FROM ratings;", conn)
conn.close()

ratings_df.head()


/var/folders/z9/k5mpzs5n7yvdr65g96bpt72m0000gn/T/ipykernel_68686/1083898168.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ratings_df = pd.read_sql("SELECT user_id, movie_id, rating FROM ratings;", conn)


,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [12]:
conn = psycopg2.connect(
    host="localhost",
    dbname="moviedb",
    user="ajinkyaambadkar",
    password="Achiever216"
)

# Load all genre mappings
movie_genres_df = pd.read_sql("""
    SELECT mg.movie_id, m.title, g.genre_name
    FROM movie_genres mg
    JOIN movies m ON mg.movie_id = m.movie_id
    JOIN genres g ON mg.genre_id = g.genre_id
""", conn)

conn.close()

movie_genres_df.head()


/var/folders/z9/k5mpzs5n7yvdr65g96bpt72m0000gn/T/ipykernel_68686/492455179.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  movie_genres_df = pd.read_sql("""


,movie_id,title,genre_name
0,242,Kolya (1996),Comedy
1,302,L.A. Confidential (1997),Crime
2,302,L.A. Confidential (1997),Film-Noir
3,302,L.A. Confidential (1997),Mystery
4,302,L.A. Confidential (1997),Thriller


In [13]:
# Create one-hot encoded genre matrix
one_hot = pd.get_dummies(movie_genres_df['genre_name'])
movies_with_genres = pd.concat([movie_genres_df[['movie_id', 'title']], one_hot], axis=1)

# Group by movie to collapse duplicates
movies_genre_matrix = movies_with_genres.groupby(['movie_id', 'title']).sum().reset_index()

movies_genre_matrix.head()



,movie_id,title,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [14]:
from sklearn.metrics.pairwise import cosine_similarity

# Drop non-feature columns
genre_features = movies_genre_matrix.drop(['movie_id', 'title'], axis=1)

# Compute cosine similarity matrix
cosine_sim = cosine_similarity(genre_features)

# Index mapping: movie title → row index
movie_indices = pd.Series(movies_genre_matrix.index, index=movies_genre_matrix['title']).drop_duplicates()


In [17]:
import psycopg2
import pandas as pd

# Connect and load movie ID → title mapping
conn = psycopg2.connect(
    host="localhost",
    dbname="moviedb",
    user="ajinkyaambadkar",
    password="Achiever216"  # replace with your actual password
)

movies_df = pd.read_sql("SELECT movie_id, title FROM movies;", conn)
conn.close()

movie_id_to_title = dict(zip(movies_df.movie_id, movies_df.title))


/var/folders/z9/k5mpzs5n7yvdr65g96bpt72m0000gn/T/ipykernel_68686/1272351306.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  movies_df = pd.read_sql("SELECT movie_id, title FROM movies;", conn)


In [18]:
from surprise import Dataset, Reader, SVD

# Re-load ratings data if needed
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings_df[['user_id', 'movie_id', 'rating']], reader)

# Train SVD model on full dataset
trainset = data.build_full_trainset()
model = SVD()
model.fit(trainset)


In [21]:
import numpy as np

alpha = 0.7  # Weight for collaborative filtering

def hybrid_recommend(user_id, top_n=10):
    # All movie IDs
    all_movie_ids = ratings_df['movie_id'].unique()
    
    # Movies already rated by the user
    rated_movies = ratings_df[ratings_df['user_id'] == user_id]['movie_id'].tolist()
    unrated_movies = [mid for mid in all_movie_ids if mid not in rated_movies]
    
    # Get user's top rated movie (used for content-based similarity)
    user_top_rated = ratings_df[ratings_df['user_id'] == user_id].sort_values(by='rating', ascending=False).head(1)
    if user_top_rated.empty:
        return ["User has no ratings. Cannot recommend."]
    
    top_movie_id = int(user_top_rated['movie_id'].iloc[0])
    top_movie_title = movie_id_to_title.get(top_movie_id, None)
    
    if top_movie_title not in movie_indices:
        return ["User's top rated movie not in similarity matrix."]
    
    top_movie_idx = movie_indices[top_movie_title]
    
    # Prepare scores list
    scores = []
    
    for mid in unrated_movies:
        # Collaborative filtering score
        cf_pred = model.predict(user_id, mid).est
        
        # Content-based score (similarity with user's top movie)
        try:
            movie_title = movie_id_to_title.get(mid, None)
            if movie_title is None or movie_title not in movie_indices:
                continue
            
            cb_score = cosine_sim[top_movie_idx][movie_indices[movie_title]].item()
        except:
            cb_score = 0
        
        # Combine both
        final_score = alpha * cf_pred + (1 - alpha) * cb_score
        scores.append((mid, final_score))
    
    # Sort and get top N
    scores = sorted(scores, key=lambda x: x[1], reverse=True)[:top_n]
    
    # Map to titles
    recommendations = []
    for mid, score in scores:
        title = movie_id_to_title.get(mid, "Unknown")
        recommendations.append(f"{title} (ID: {mid}) - Score: {score:.2f}")
    
    return recommendations


In [22]:
hybrid_recommend(user_id=100)


["Schindler's List (1993) (ID: 318) - Score: 3.18",
 'Silence of the Lambs, The (1991) (ID: 98) - Score: 3.10',
 'Casablanca (1942) (ID: 483) - Score: 3.10',
 'Shawshank Redemption, The (1994) (ID: 64) - Score: 3.09',
 'Sling Blade (1996) (ID: 223) - Score: 3.01',
 'Braveheart (1995) (ID: 22) - Score: 2.99',
 'Pather Panchali (1955) (ID: 1449) - Score: 2.98',
 'Close Shave, A (1995) (ID: 408) - Score: 2.97',
 'Jean de Florette (1986) (ID: 165) - Score: 2.97',
 'Empire Strikes Back, The (1980) (ID: 172) - Score: 2.97']

In [23]:
import psycopg2
from datetime import datetime

def save_recommendations_to_db(user_id, recs):
    conn = psycopg2.connect(
        host="localhost",
        dbname="moviedb",
        user="ajinkyaambadkar",
        password="Achiever216"
    )
    cur = conn.cursor()

    for rec in recs:
        # Extract ID and score
        movie_id = int(rec.split("ID: ")[1].split(")")[0])
        score = float(rec.split("Score: ")[1])

        cur.execute("""
            INSERT INTO recommendations (user_id, movie_id, score)
            VALUES (%s, %s, %s)
            ON CONFLICT (user_id, movie_id) DO UPDATE SET
                score = EXCLUDED.score,
                recommended_on = CURRENT_TIMESTAMP;
        """, (user_id, movie_id, score))

    conn.commit()
    cur.close()
    conn.close()
    print(f"✅ Stored {len(recs)} recommendations for user {user_id}")


In [24]:
recs = hybrid_recommend(user_id=100)
save_recommendations_to_db(user_id=100, recs=recs)

✅ Stored 10 recommendations for user 100


In [30]:
import os
print(os.getcwd())


/Users/ajinkyaambadkar/Projects/movie-recommendation-system/notebooks


In [31]:
import sys
import os
import pandas as pd

# 🔍 Adjust this based on where your notebook is
sys.path.append(os.path.abspath("../src"))  # if you're inside /notebooks

from utils.data_queries import get_top_movies

print(get_top_movies())


                                               title  avg_rating
0                      Someone Else's America (1995)         5.0
1  Entertaining Angels: The Dorothy Day Story (1996)         5.0
2               Saint of Fort Washington, The (1993)         5.0
3                          Santa with Muscles (1996)         5.0
4                     They Made Me a Criminal (1939)         5.0
5                      Great Day in Harlem, A (1994)         5.0
6                                 Prefontaine (1997)         5.0
7                               Aiqing wansui (1994)         5.0
8                                    Star Kid (1997)         5.0
9         Marlene Dietrich: Shadow and Light (1996)          5.0


/Users/ajinkyaambadkar/Projects/movie-recommendation-system/src/utils/data_queries.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, params=(limit,))
